Section 1

(core) Animating fractal constructions gives us direct insight into how the function generating the fractal operates. You are given a pygame animation function draw_sierpinski that draws the Sierpinski triangle. Using this function the user is able to choose the depth of the triangle and to stop and start the animation. You should develop this function so that the user is also able to change the speed of the animation. You should also add colours to the triangle drawing.

We start by importing some code from document given.

In [1]:
import random, pygame, os

pygame 2.6.1 (SDL 2.28.4, Python 3.13.5)
Hello from the pygame community. https://www.pygame.org/contribute.html


Content: making the triangle

In [10]:
def color_at(level: int, depth_max: int) -> tuple[int, int, int]:
    t = level / max(1, depth_max)
    r = int(0   * (1 - t) + 200 * t)
    g = int(120 * (1 - t) +   0 * t)
    b = int(255 * (1 - t) + 200 * t)
    return r, g, b

def make_sierpinski(depth, triangle, triangle_list, level=0, depth_max=None):
    if depth_max is None:
        depth_max = depth


    (x0, y0), (x1, y1), (x2, y2) = triangle


    if depth == 1:
        col = color_at(level, depth_max)
        verts = [(int(x0), int(y0)), (int(x1), int(y1)), (int(x2), int(y2))]
        triangle_list.append((verts, col))
        return

    ABm = ((x0 + x1) / 2.0, (y0 + y1) / 2.0)
    BCm = ((x1 + x2) / 2.0, (y1 + y2) / 2.0)
    CAm = ((x2 + x0) / 2.0, (y2 + y0) / 2.0)


    make_sierpinski(depth-1, ((x0, y0), ABm, CAm), triangle_list, level+1, depth_max)
    make_sierpinski(depth-1, (ABm, (x1, y1), BCm), triangle_list, level+1, depth_max)
    make_sierpinski(depth-1, (CAm, BCm, (x2, y2)), triangle_list, level+1, depth_max)

Content: Animation

In [11]:
SPEED_FACTOR = 4
def draw_sierpinski(depth=6):
    '''
    Function that draws the Sierpinski triangle as an animation. 
    The depth of the triangle (recursion) can be adjusted by entering 
    a depth integer value (in [1,10]) as a parameter. 
    For example: python sierpinski.py 8 
    '''
    
    dimensions = (900, 862)
    backgroundColour = (255,255,255)
    blue, black = (0,0,255), (0,0,0)
    # This is the overall outline triangle
    master_triangle = ((50,800),(850,800),(450,62))
    min_depth, max_depth = 1, 10
    clock = pygame.time.Clock()
    warning = "Depth must be an integer in the interval [1,10]"

    if depth < min_depth: 
        depth = min_depth
        print(warning)
        print("Using depth {}".format(min_depth))
    if depth > max_depth: 
        depth = max_depth
        print(warning)
        print("Using depth {}".format(max_depth))

    global SPEED_FACTOR
    try:
        speed_factor = int(SPEED_FACTOR)
    except NameError:
        speed_factor = 4
    # Defines the speed of the animation (see the animation loop) 
    frames_per_second = 10 * speed_factor
    # Make a list of all the triangle vertex coordinates of the given 
    # depth (in make_sierpinski we process  depth to work down to 1)
    triangle_list = []
    make_sierpinski(depth,master_triangle,triangle_list)

    # Initialise pygame and the screen display object and title
    pygame.init()
    screen = pygame.display.set_mode(dimensions)
    # Put the title and instructions for the animation in the title bar of the animation.
    caption = 'Sierpinski Triangle            '
    caption += '(1)  \'Space\' to start or pause    '
    caption += '(2)  Further keystroke instruction here?'
    pygame.display.set_caption(caption)

    # Initialise the display 
    screen.fill(backgroundColour)
    pygame.display.flip()

    # Total number of triangles to be drawn 
    number_of_triangles = len(triangle_list)
    index = 0
    draw_triangle = False
    keep_running = True

    # Animation loop 
    while keep_running:
        for event in pygame.event.get():
            # Exit (at end of this iteration) using quit (e.g Ctrl-q or red button)
            if event.type == pygame.QUIT:
                keep_running = False
            # Start and pause the animation with the space key 
            elif event.type == pygame.KEYDOWN and event.key == pygame.K_SPACE:
                draw_triangle  = not draw_triangle 

        # Keep draw next triangle with index 'index' if not told to pause and not complete
        if draw_triangle and index  < number_of_triangles:
            verts, col = triangle_list[index] 
            pygame.draw.polygon(screen, col,  verts, 0)
            pygame.draw.polygon(screen, (0,0,0), verts, 1)
            pygame.display.update()
            index += 1

             # Pause time before next iteration starts: one clock tick  
            clock.tick(frames_per_second)
    pygame.quit()
    return None

A test function

In [12]:
draw_sierpinski(5)

Let user enter the depth

In [5]:
def run_sierpinski(): 
    min_depth, max_depth = 1, 10
    default_depth = 6
    # Get the depth from the user 
    try:
        # If either of the following lines failsthen the body of the except statement is run
        depth = int(input("Enter a depth (from {} to {}): ".format(min_depth,max_depth)))
        assert min_depth <= depth <= max_depth
    except:
        print("There was a problem with your input.", end = " ") 
        print("Using default depth:{}".format(default_depth))
        depth = default_depth
    # Now run the animation with the depth input by the user
    draw_sierpinski(depth) 
    return None

A test function

In [6]:
run_sierpinski()


Enter a depth (from 1 to 10):  6


You are asked to develop the function draw_sierpinski so that the user is also able to change the speed of the animation. You should also add colours to the triangle drawing (either in the same or a different script).

Firstly we add a choice where user could change the speed of the animation.

In [7]:
def speed_sierpinski():
    min_speed, max_speed = 1, 10
    default_speed = 5
    global SPEED_FACTOR       

    try:
        s = int(input(f"Enter a speed (from {min_speed} to {max_speed}): "))
        assert min_speed <= s <= max_speed
        SPEED_FACTOR = s        
    except Exception:
        print("There was a problem with your input. Using default:", default_speed)
        SPEED_FACTOR = default_speed

    run_sierpinski()
    return None

In [9]:
speed_sierpinski()

Enter a speed (from 1 to 10):  10
Enter a depth (from 1 to 10):  4


Meanwhile, we add colours to triangle drawing. We do this by adding code to make_sierpinski function so that color is added in the process of contructing instead of in other stages. Note that the make_sierpinski function is different from that in the document given: it adds a function dealing color.

Section2

 (core) Fractals can simulate shapes found in the natural word. One simple example of
this is the construction of a recursively defined tree. The tree of depth 1 is just a trunk
with three straight branches. Then given the tree T of depth n the tree of depth n + 1
is the tree T where every branch has been replaced by a tree of depth 1.You should develop a pygame animation function that constructs a similar recursive tree.
The user should be able to choose the depth of the tree and should be able to start and
stop the animation and control its speed. The tree should be coloured with the trunk and
branches being brown and the leaves (i.e. the last level of branches) being green.